In [71]:
# Load your IBM Q account
from qiskit import IBMQ

IBMQ.save_account('0c5b2835672517fb24d1ecd812570bac90ef6628b742d93906192c0a587d2ef0511a7e51bb9bc3c4530949c2ef46a084a88054a29e0a5ff2ad84c82fbad76cb7', overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
realibmlagos_backend = provider.get_backend('ibm_perth')

ibmqfactory.load_account:WARNING:2023-11-26 22:24:14,741: Credentials are already in use. The existing account in the session will be replaced.


In [69]:
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

# Load your IBM Q account
IBMQ.load_account()

# Get the provider for IBM Q
provider = IBMQ.get_provider(hub='ibm-q')

# Filter backends that are operational and have between 3 and 7 qubits
backends = provider.backends(filters=lambda x: 3 <= x.configuration().n_qubits <= 7 
                             and not x.configuration().simulator 
                             and x.status().operational==True)

# Find the least busy backend among the filtered backends
least_busy_backend = least_busy(backends)
print("Least busy backend suitable for a 3-qubit system:", least_busy_backend)


ibmqfactory.load_account:WARNING:2023-11-26 22:23:37,282: Credentials are already in use. The existing account in the session will be replaced.


Least busy backend suitable for a 3-qubit system: ibm_perth


# Error Detection With Initial Syndrome Pairs

In [87]:
from qiskit import QuantumCircuit, transpile
from qiskit.tools.monitor import job_monitor

# Error Detection Circuit
qc_detect = QuantumCircuit(5, 2)
qc_detect.initialize([1, 0], 0)
qc_detect.cx(0, 1)
qc_detect.cx(0, 2)
qc_detect.barrier()
qc_detect.x(2)  # Introduce a bit flip error on the third qubit
qc_detect.barrier()
qc_detect.cx(0, 3)
qc_detect.cx(2, 3)
qc_detect.cx(0, 4)
qc_detect.cx(1, 4)
qc_detect.measure(3, 0)
qc_detect.measure(4, 1)

# Transpile for the real quantum device
transpiled_qc_detect = transpile(qc_detect, realibmlagos_backend)

# Specify the number of shots
num_shots = 1024

# Submitting the job to the real quantum device
job_detect = realibmlagos_backend.run(transpiled_qc_detect,shots=num_shots)
job_monitor(job_detect)

# Retrieve and print the result
result_detect = job_detect.result()
error_counts_detect = result_detect.get_counts(qc_detect)
print("Error Detection Counts on Real Quantum Computer:")
print(error_counts_detect)


Job Status: job has successfully run
Error Detection Counts on Real Quantum Computer:
{'00': 100, '11': 69, '10': 8, '01': 847}


# Error Detection Circuit With the Different Syndrome Pairs

In [42]:
# Error Detection Circuit
qc_detect = QuantumCircuit(5, 2)

# Initialize the first qubit to |1>
qc_detect.initialize([1, 0], 0)

# Entangle the first three qubits
qc_detect.cx(0, 1)
qc_detect.cx(0, 2)
qc_detect.barrier()

# Introduce a bit flip error on the third qubit
qc_detect.x(2)
qc_detect.barrier()

# Set up ancilla qubits for error detection
# First ancilla qubit for parity check between qubit 0 and qubit 1
qc_detect.cx(0, 3)
qc_detect.cx(1, 3)

# Second ancilla qubit for parity check between qubit 1 and qubit 2
qc_detect.cx(1, 4)
qc_detect.cx(2, 4)

# Measure the ancilla qubits
qc_detect.measure(3, 0) # Measure first ancilla qubit
qc_detect.measure(4, 1) # Measure second ancilla qubit

# Transpile for the real quantum device
transpiled_qc_detect = transpile(qc_detect, realibmlagos_backend)

# Specify the number of shots
num_shots = 1024

# Submitting the job to the real quantum device
job_detect = realibmlagos_backend.run(transpiled_qc_detect, shots=num_shots)
job_monitor(job_detect)

# Retrieve and print the result
result_detect = job_detect.result()
error_counts_detect = result_detect.get_counts(qc_detect)
print("Error Detection Counts on Real Quantum Computer With Different Syndrome Pairs:")
print(error_counts_detect)


Job Status: job has successfully run
Error Detection Counts on Real Quantum Computer With Different Syndrome Pairs:
{'01': 33, '10': 880, '00': 73, '11': 38}


# Error Correction Circuit For the Most Frequent Syndrome '01' means Third Bit From Initial Syndrome Pairs

In [79]:
most_common_syndrome = max(error_counts_detect, key=error_counts_detect.get)
most_common_syndrome

'01'

In [ ]:
# Analyze the detection results to decide on the correction
most_common_syndrome = max(error_counts_detect, key=error_counts_detect.get)

# Error Correction Circuit
qc_correct = QuantumCircuit(5, 2)

# Apply the correction based on the syndrome
if most_common_syndrome == '01':
    qc_correct.x(2)  # Apply X gate to the third qubit for syndrome '10'

# Combine the detection and correction circuits
qc_combined_correction = qc_detect.compose(qc_correct)

# Transpile and run the combined correction circuit
transpiled_qc_combined_correction = transpile(qc_combined_correction, realibmlagos_backend)
job_combined_correction = realibmlagos_backend.run(transpiled_qc_combined_correction, shots=1024)
job_monitor(job_combined_correction)
result_combined_correction = job_combined_correction.result()
error_counts_combined_correction = result_combined_correction.get_counts(qc_combined_correction)
print("Error Counts after Correction on Real Quantum Computer For the Most Frequent Syndrome '10':")
print(error_counts_combined_correction)


In [ ]:
# Error Counts after Correction on Real Quantum Computer For the Most Frequent Syndrome '01':
 {'00': 92, '01': 820, '11': 94, '10': 18}

# # Error Correction Circuit For the Most Frequent Syndrome '10' means Third Bit From Different Syndrome Pairs with different Backend 'ibm_lagos'

In [66]:
# Analyze the detection results to decide on the correction
most_common_syndrome = max(error_counts_detect, key=error_counts_detect.get)

# Error Correction Circuit
qc_correct = QuantumCircuit(5, 2)

# Apply the correction based on the syndrome
if most_common_syndrome == '10':
    qc_correct.x(2)  # Apply X gate to the third qubit for syndrome '10'

# Combine the detection and correction circuits
qc_combined_correction = qc_detect.compose(qc_correct)

# Transpile and run the combined correction circuit
transpiled_qc_combined_correction = transpile(qc_combined_correction, realibmlagos_backend)
job_combined_correction = realibmlagos_backend.run(transpiled_qc_combined_correction, shots=1024)
job_monitor(job_combined_correction)
result_combined_correction = job_combined_correction.result()
error_counts_combined_correction = result_combined_correction.get_counts(qc_combined_correction)
print("Error Counts after Correction on Real Quantum Computer For the Most Frequent Syndrome '10':")
print(error_counts_combined_correction)


Job Status: job has successfully run
Error Counts after Correction on Real Quantum Computer For the Most Frequent Syndrome '10':
{'01': 29, '11': 43, '10': 837, '00': 115}


In [75]:
most_common_syndrome = max(error_counts_detect, key=error_counts_detect.get)
most_common_syndrome

'10'